In [1]:
"""
Extract Active Fire Detections (AFDs) from MODIS and VIIRS within "aspen fires"

Author: maxwell.cook@colorado.edu
"""

import os, time, glob, sys
import geopandas as gpd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore") # suppresses annoying geopandas warning

# Custom functions
sys.path.append(os.path.join(os.getcwd(),'code/'))
from __functions import *

proj = 'EPSG:5070'

maindir = '/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/'
projdir = os.path.join(maindir, 'aspen-fire/Aim2/')

print("Ready to go !")

Ready to go !


In [ ]:
# Load the fire data.

In [2]:
aspen_events_fp = os.path.join(projdir,'data/spatial/mod/NIFC/nifc-interagency_2018_to_2023-EVT-aspen.gpkg')
aspen_events = gpd.read_file(aspen_events_fp)
aspen_events.columns

Index(['OBJECTID', 'na_l3name', 'MAP_METHOD', 'GEO_ID', 'IRWINID', 'FIRE_YEAR',
       'INCIDENT', 'FEATURE_CA', 'GIS_ACRES', 'pct_aspen', 'geometry'],
      dtype='object')

In [3]:
# Create a fire perimeter buffer to extract FRP observations within 1km of bounds.
bounds = create_bounds(aspen_events, by_bounds=True, buffer=1000)
bounds = bounds.set_crs(proj, allow_override=True) # ensure correct crs
bounds = bounds[['OBJECTID','FIRE_YEAR','INCIDENT','geometry']] 
bounds['FIRE_YEAR'] = bounds['FIRE_YEAR'].astype(int)
bounds.head()

,OBJECTID,FIRE_YEAR,INCIDENT,geometry
0,6,2018,DEVIL CREEK,"POLYGON ((-981360.461 1644805.834, -981864.344..."
1,8,2019,577,"POLYGON ((-1003641.705 1627304.106, -1004580.3..."
2,12,2018,SERVICEBERRY,"POLYGON ((-955842.719 1623123.110, -956568.269..."
3,14,2018,416,"POLYGON ((-1031771.042 1658048.676, -1047969.7..."
4,23,2020,LOADING PEN,"POLYGON ((-1071014.573 1681756.433, -1071765.1..."


In [4]:
bounds.crs

<Projected CRS: EPSG:5070>
Name: NAD83 / Conus Albers
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: United States (USA) - CONUS onshore - Alabama; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming.
- bounds: (-124.79, 24.41, -66.91, 49.38)
Coordinate Operation:
- name: Conus Albers
- method: Albers Equal Area
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [ ]:
# Load the Active Fire Detection (AFD) data

AFDs from MODIS Collection 6.1 (1km) and the Suomi National Polar-Orbiting Partnership (VIIRS S-NPP 375m) were obtained from the NASA FIRMS (https://firms.modaps.eosdis.nasa.gov/download/) between 2018-2023 in the western US. The S-NPP AFDs include an archive shapefile and a NRT shapefile for Sept 2022-->

In [5]:
# Gather the archive and NRT S-NPP AFDs.
snpp_fp = os.path.join(projdir,'data/spatial/raw/NASA-FIRMS/DL_FIRE_SV-C2_476784/')
vects = glob.glob(snpp_fp+"*.shp")
print([os.path.basename(v) for v in vects])

['fire_nrt_SV-C2_476784.shp', 'fire_archive_SV-C2_476784.shp']


In [6]:
# Merge the NRT and archive vintages
archive = gpd.read_file([v for v in vects if "archive" in v][0]).to_crs(proj)
nrt = gpd.read_file([v for v in vects if "nrt" in v][0]).to_crs(proj)
snpp = pd.concat([archive, nrt], ignore_index=True)
snpp.head()

,LATITUDE,LONGITUDE,BRIGHTNESS,SCAN,TRACK,ACQ_DATE,ACQ_TIME,SATELLITE,INSTRUMENT,CONFIDENCE,VERSION,BRIGHT_T31,FRP,DAYNIGHT,TYPE,geometry
0,36.622398,-119.958420,319.25,0.45,0.63,2018-01-01,0854,N,VIIRS,n,1,276.46,1.93,N,0.0,POINT (-2100437.814 1774627.392)
1,48.166241,-102.698212,323.92,0.38,0.36,2018-01-01,0854,N,VIIRS,n,1,244.01,3.01,N,0.0,POINT (-502299.189 2814197.682)
2,48.152390,-102.697792,325.34,0.38,0.36,2018-01-01,0854,N,VIIRS,n,1,247.16,2.07,N,0.0,POINT (-502375.241 2812672.753)
3,47.879730,-102.706543,320.74,0.38,0.36,2018-01-01,0854,N,VIIRS,n,1,245.34,3.00,N,0.0,POINT (-505150.481 2782728.231)
4,44.249660,-104.516678,327.49,0.38,0.36,2018-01-01,0854,N,VIIRS,n,1,251.79,5.84,N,0.0,POINT (-677212.174 2392519.636)


In [7]:
# Set up a dictionary to store both MODIS and VIIRS AFDs
modis_fp = os.path.join(projdir,'data/spatial/raw/NASA-FIRMS/DL_FIRE_M-C61_476781/')

# Store these in a dictionary
afds = {
    "MOD61": gpd.read_file(modis_fp).to_crs(proj),
    "SNPP": snpp
}

afds['MOD61'].head()

,LATITUDE,LONGITUDE,BRIGHTNESS,SCAN,TRACK,ACQ_DATE,ACQ_TIME,SATELLITE,INSTRUMENT,CONFIDENCE,VERSION,BRIGHT_T31,FRP,DAYNIGHT,TYPE,geometry
0,44.3762,-119.1190,312.5,1.2,1.1,2018-01-01,0608,Terra,MODIS,85,6.03,269.1,21.9,N,0,POINT (-1819292.701 2598646.798)
1,32.1679,-107.7655,322.3,1.1,1.0,2018-01-01,1809,Terra,MODIS,77,6.03,289.1,19.0,D,0,POINT (-1101321.656 1078752.715)
2,32.1643,-107.7433,318.4,1.1,1.0,2018-01-01,1809,Terra,MODIS,74,6.03,287.9,14.9,D,0,POINT (-1099303.652 1078097.526)
3,31.7188,-102.0085,303.8,1.4,1.2,2018-01-01,1944,Aqua,MODIS,57,6.03,279.4,12.6,D,0,POINT (-566656.554 978378.293)
4,38.8238,-122.6829,302.0,2.7,1.6,2018-01-01,1946,Terra,MODIS,50,6.03,280.2,26.9,D,0,POINT (-2264952.497 2075236.763)


In [8]:
afds['MOD61'].crs

<Projected CRS: EPSG:5070>
Name: NAD83 / Conus Albers
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: United States (USA) - CONUS onshore - Alabama; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming.
- bounds: (-124.79, 24.41, -66.91, 49.38)
Coordinate Operation:
- name: Conus Albers
- method: Albers Equal Area
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [ ]:
# Extract the AFDs within fire bounds.

In [11]:
afd_events = {} # to store the filtered data
# Loop through sensors (MODIS and VIIRS)
for satellite, afd in afds.items():
    print(f"Processing: {satellite}")
    
    afd['VID'] = afd.index # add a unique ID
    
    # Remove low confidence observations
    try:
        afd = afd[afd['CONFIDENCE'] != 'l']
    except KeyError as e:
        print(f"KeyError: {e}")

    if afd.crs != bounds.crs:
        afd = afd.to_crs(bounds.crs)  # ensure the crs matches

    # Extract within fire bounds
    afd_ = gpd.sjoin(afd, bounds, how='inner', predicate='within')
    afd_.drop(columns=['index_right'], inplace=True)
    
    # Tidy the date columns for matching
    afd_['ACQ_DATE'] = pd.to_datetime(afd_['ACQ_DATE'])
    afd_['ACQ_MONTH'] = afd_['ACQ_DATE'].dt.month.astype(int)
    afd_['ACQ_YEAR'] = afd_['ACQ_DATE'].dt.year.astype(int)
    
    # Create a datetime object
    afd_.loc[:, 'ACQ_DATETIME'] = afd_.apply(lambda row: convert_datetime(row['ACQ_DATE'], row['ACQ_TIME']), axis=1)

    # Filter for matching fire year
    afd_f = afd_[afd_['ACQ_YEAR'] == afd_['FIRE_YEAR']]

    # Drop any duplicates
    dups = afd_f.duplicated(subset='VID').sum()
    if dups > 0:
        print(f"Be aware ! Duplicates found ! {dups} to be exact ...")
        # afd_f = afd_f.drop_duplicates(subset='VID', keep='first')
    
    print(f"\tThere are [{len(afd_f)}] high or nominal confidence detections within fire bounds.")

    # Overwrite the dictionary items
    afd_events[satellite] = afd_f

    # Save the AFD observation in fire bounds
    out_fp = os.path.join(projdir,f'data/spatial/mod/AFD/{satellite}-afd_fired-aspen_2018_to_2023.gpkg')
    afd_f.to_file(out_fp)
    print(f'Saved to {out_fp}\n')
    
    del afd, afd_, afd_f
    gc.collect()

print("\nProcessing complete !")

Processing: MOD61
Be aware ! Duplicates found ! 1405 to be exact ...
	There are [30214] high or nominal confidence detections within fire bounds.
Saved to /Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim2/data/spatial/mod/AFD/MOD61-afd_fired-aspen_2018_to_2023.gpkg

Processing: SNPP
Be aware ! Duplicates found ! 4955 to be exact ...
	There are [125503] high or nominal confidence detections within fire bounds.
Saved to /Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim2/data/spatial/mod/AFD/SNPP-afd_fired-aspen_2018_to_2023.gpkg


Processing complete !


In [ ]:
# Handle duplicates seperately

In [ ]:
afd_events['MOD61'].columns

In [ ]:
# Create the AFD "detection buffer" for both MODIS and VIIRS

In [ ]:
afd_events_plots = {} # to store the buffered observations
for satellite, afd in afd_events.items():
    print(f"Creating buffer for {satellite} observations.")

    afd_ = afd.copy()
    if satellite == 'MOD61':
        afd_.geometry = afd_.geometry.buffer(500, cap_style=3) # square buffer half of pixel size
    elif satellite == 'SNPP':
        afd_.geometry = afd_.geometry.buffer(187.5, cap_style=3) # square buffer half of pixel size

    afd_events_plots[satellite] = afd_ # add to the new dictionary

    # Save out
    out_fp = os.path.join(projdir,f'data/spatial/mod/AFD/{satellite}-afd_fired-aspen_2018_to_2023_buffer.gpkg')
    afd_.to_file(out_fp)
    print(f'Saved to {out_fp}\n')

    del afd_
    gc.collect()

In [ ]:
afd_events_plots['MOD61'].columns

In [ ]:
# Gather bounds for individual fires/AFDs

In [ ]:
buffer = 1000 # meters
afd_fire_bounds = {}
for satellite, afd in afd_events_plots.items():
    print(f"Creating bounds for {satellite} observations.")
    
    # Group by fire ID
    grouped = afd.groupby('OBJECTID')
    bounds_gdf = gpd.GeoDataFrame(columns=['OBJECTID', 'FIRE_YEAR', 'geometry'], crs=afd.crs)
    # Iterate over each fire group
    for fire_id, group in grouped:
        # Calculate the total bounds (minx, miny, maxx, maxy) for all geometries in the group
        bounds = box(*group.total_bounds)
        # Apply buffer if specified
        if buffer is not None:
            bounds = bounds.buffer(buffer)
        # Extract the ig_year (assuming it's the same for all AFD in the group)
        ig_year = group['FIRE_YEAR'].iloc[0]
        # Create a GeoDataFrame for this fire with the bounding box, id, and ig_year
        fire_gdf = gpd.GeoDataFrame({'OBJECTID': [fire_id], 'FIRE_YEAR': [ig_year], 'geometry': [bounds]}, crs=afd.crs)
        # Append the result to the GeoDataFrame
        bounds_gdf = pd.concat([bounds_gdf, fire_gdf], ignore_index=True)

    # Store the result
    afd_fire_bounds[satellite] = bounds_gdf

    # Save out the bounds to a file
    out_fp = os.path.join(projdir, f'data/spatial/mod/AFD/{satellite}-afd_fired-aspen_2018_to_2023_bounds.gpkg')
    bounds_gdf.to_file(out_fp)
    print(f"Saved to {out_fp}\n")

    gc.collect()

In [ ]:
# Combine the MODIS and VIIRS bounds into a single GeoDataFrame
combined = pd.concat([afd_fire_bounds['MOD61'], afd_fire_bounds['SNPP']], ignore_index=True)
# Dissolve the geometries by fire ID ('OBJECTID') to combine geometries for each fire
combined = combined.dissolve(by='OBJECTID')

In [ ]:
# Join to the fire season info from '_west-fire-season-length.ipynb'

In [ ]:
# Save out the bounds to a file.
out_fp = os.path.join(projdir, f'data/spatial/mod/AFD/all-afd_fired-aspen_2018_to_2023_bounds.gpkg')
combined.to_file(out_fp)
print(f"Saved to {out_fp}\n")

In [ ]:
gc.collect()